# Libraries

In [ ]:
from google.colab import drive 
drive.mount('/content/drive') 

In [ ]:
from scipy import misc, ndimage, signal
import numpy 
import numpy as np
import random 
import ntpath
import os
import pandas as pd 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from tensorflow.keras import backend
from keras.layers import Lambda, Layer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Reshape
from keras import optimizers
from keras.utils import np_utils
from keras import backend as K
from keras import regularizers
from time import time
from keras.models import load_model
import keras
import datetime
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.metrics import categorical_crossentropy
from keras import optimizers 
from keras import regularizers
from keras.layers import Lambda, Layer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.initializers import Constant, RandomNormal, glorot_normal
from keras.layers.core import Reshape
from keras import optimizers
from keras.utils import np_utils
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.callbacks import TensorBoard
from time import time
from random import randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import random
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from numpy import *
from PIL import Image
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import tensorflow as tf
import sys
import glob
from scipy import misc, ndimage, signal
import numpy
import random
import ntpath
import skimage
from skimage.util.shape import view_as_blocks, view_as_windows
import tensorflow as tf
from tensorflow import clip_by_value
import sys
import glob
from scipy import misc, ndimage, signal
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
from keras.layers import Lambda, Layer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, ReLU
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Reshape
from keras.utils import np_utils
from keras import backend as K
import pandas as pd
from tensorflow import clip_by_value
import time as tm
from tensorflow.keras.layers import Concatenate, Lambda
import numpy as np
import time
import time as tm
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.base import BaseEstimator
import keras
from keras.models import Model
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

# Loading database

In [ ]:
X_train_N = np.load('.../Seed#/X_Neg_Train_224_Seed2.npy')
X_train_P = np.load('.../Seed#/X_Pos_Train_224_Seed2.npy')

X_test_N = np.load('.../Seed#/X_Neg_Test_224_Seed2.npy')
X_test_P = np.load('.../Seed#/X_Pos_Test_224_Seed2.npy') 

In [ ]:
y_train_N=np.array([[1,0]]*len(X_train_N))
y_test_N =np.array([[1,0]]*len(X_test_N))

y_train_P=np.array([[0,1]]*len(X_train_P))
y_test_P =np.array([[0,1]]*len(X_test_P))

X_train=np.concatenate((X_train_N,X_train_P))
X_test =np.concatenate((X_test_N,X_test_P))
y_train=np.concatenate((y_train_N,y_train_P))
y_test =np.concatenate((y_test_N,y_test_P)) 

In [ ]:
y_train_N.shape, y_train_P.shape 

In [ ]:
y_test_N.shape, y_test_P.shape 

In [ ]:
X_train.shape 

In [ ]:
X_test.shape 

In [ ]:
print(X_train.min(),X_test.min())
print(X_train.max(),X_test.max()) 

In [ ]:
print(X_train.shape[0],X_train_N.shape[0],X_train_P.shape[0]) 
print(X_test.shape[0], X_test_N.shape[0], X_test_P.shape[0]) 

# Functions used to preprocess the data 

In [ ]:
def samplewise_preprocessing(images,labels):
    filtered_labels=[]
    processed_images = []
    means = []
    stds = []
    for i in range(images.shape[0]):
        mean = np.mean(images[i]) 
        std = np.std(images[i]) 
        if std!=0 and mean != 0:
            means.append(mean)
            stds.append(std)
            processed_images.append((images[i]-mean)/std)
            filtered_labels.append(labels[i])
    
    return np.array(processed_images),np.array(filtered_labels), np.mean(means), np.mean(stds)

def featurewise_preprocessing(images, mean, std):
    processed_images = np.zeros_like(images, dtype=np.float32)
    for i in range(images.shape[0]):
        processed_images[i] = (images[i]-mean)/std
    return processed_images

def min_max_preprocessing(images,labels):
    filtered_labels=[]
    processed_images = []
    for i in range(len(images)):
        maxi=np.max(images[i])
        mini=np.min(images[i])
        if maxi-mini != 0:
          processed_images.append((images[i]-mini)/(maxi-mini))
          filtered_labels.append(labels[i])
    return np.array(processed_images),np.array(filtered_labels)

def convert_n_CH(train_images,test_images,mode='gray'):
    if mode=='gray':
        converted_train=np.expand_dims(train_images,axis=3)
        converted_test=np.expand_dims(test_images,axis=3)
    elif mode=='rgb':
        converted_train=np.array([cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in train_images])
        converted_test=np.array([cv2.cvtColor(img,cv2.COLOR_GRAY2BGR) for img in test_images])

    return converted_train,converted_test

# Functions used to evaluate the models

In [ ]:
def metrics(Y_true,predictions):
    """
    Function to print the performance metrics.

    Inputs
    Y_true: Ground truth labels
    predictions: Predicted labels

    Outputs
    Accuracy, F1 Score, Recall, Precision, Classification report, Confusion matrix
    """

    print('Accuracy:', accuracy_score(Y_true, predictions))
    print('F1 score:', f1_score(Y_true, predictions,average='weighted'))
    print('Recall:', recall_score(Y_true, predictions,average='weighted'))
    print('Precision:', precision_score(Y_true, predictions, average='weighted'))
    print('\n Clasification report:\n', classification_report(Y_true, predictions))
    print('\n Confusion matrix:\n',confusion_matrix(Y_true, predictions))

    snn_cm = confusion_matrix(Y_true, predictions)
    snn_df_cm = pd.DataFrame(snn_cm, range(2), range(2))  
    plt.figure(figsize = (9,5))  
    sn.set(font_scale=1.4)
    sn.heatmap(snn_df_cm, annot=True, annot_kws={"size": 12}, fmt="d")  
    plt.show()  

def metrics_1(model,X_test,y_test):
    predictions=model.predict(X_test)
    predictions = np.argmax(predictions, axis=-1)
    y_test=np.argmax(y_test,axis=-1)
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('F1 score:', f1_score(y_test, predictions))
    print('Recall:', recall_score(y_test, predictions))
    print('Precision:', precision_score(y_test, predictions))
    print('\n clasification report:\n', classification_report(y_test, predictions))
    print('\n confusion matrix:\n',confusion_matrix(y_test, predictions))
    print("Verdaderos positivos:",get_true_pos(y_test,predictions))
    print("Verdaderos negativos:",get_true_neg(y_test,predictions))
    print("Falsos negativos:",get_false_neg(y_test,predictions))
    print("Falsos Positivos:",get_false_pos(y_test,predictions))
    print("Sensibilidad:",get_sensitivity(y_test,predictions))
    print("Especificidad:",get_specificity(y_test,predictions))

def get_true_pos(y, pred, th=0.5):

    """
    Function to calculate the total of true positive (TP) predictions.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """

    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 1))

def get_true_neg(y, pred, th=0.5):

    """
    Function to calculate the total of true negative (TN) predictions.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """

    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 0))

def get_false_neg(y, pred, th=0.5):

    """
    Function to calculate the total of false negative (FN) predictions.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """
    
    pred_t = (pred > th)
    return np.sum((pred_t == False) & (y == 1))

def get_false_pos(y, pred, th=0.5):

    """
    Function to calculate the total of false positive (FP) predictions.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """

    pred_t = (pred > th)
    return np.sum((pred_t == True) & (y == 0))

def print_confidence_intervals(class_labels, statistics):

    """
    Function to calculate the confidence interval (5%-95%).
    
    Inputs
    class_labels: List with class names
    statistics: 

    Outputs
    Returns DataFrame with confidence intervals for each class
    """

    df = pd.DataFrame(columns=["Mean AUC (CI 5%-95%)"])
    for i in range(len(class_labels)):
        mean = statistics.mean(axis=1)[i]
        max_ = np.quantile(statistics, .95, axis=1)[i]
        min_ = np.quantile(statistics, .05, axis=1)[i]
        df.loc[class_labels[i]] = ["%.2f (%.2f-%.2f)" % (mean, min_, max_)]
    return df

def get_roc_curve(gt, pred, target_names):

    """
    Function to plot the ROC curve.
    
    Inputs
    gt: Ground truth labels
    pred: Predicted labels
    target_names: List with class names 
    """

    for i in range(len(target_names)):
        auc_roc = roc_auc_score(gt[:, i], pred[:, i])
        label = target_names[i] + " AUC: %.3f " % auc_roc
        xlabel = "False positive rate"
        ylabel = "True positive rate"
        a, b, _ = roc_curve(gt[:, i], pred[:, i])
        plt.figure(1, figsize=(7, 7))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(a, b, label=label)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1), fancybox=True, ncol=1)

def plot_calibration_curve(y, pred,class_labels):

    """
    Function to plot the calibration curve.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    class_labels: List with class names 
    """

    plt.figure(figsize=(20, 20))
    for i in range(len(class_labels)):
        plt.subplot(4, 4, i + 1)
        fraction_of_positives, mean_predicted_value = calibration_curve(y[:,i], pred[:,i], n_bins=20)
        plt.plot([0, 1], [0, 1], linestyle='--')
        plt.plot(mean_predicted_value, fraction_of_positives, marker='.')
        plt.xlabel("Predicted Value")
        plt.ylabel("Fraction of Positives")
        plt.title(class_labels[i])
    plt.tight_layout()
    plt.show()

def get_accuracy(y, pred, th=0.5):

    """
    Function to calculate the accuracy.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """

    accuracy = 0.0
    TP = get_true_pos(y, pred, th)
    FP = get_false_pos(y, pred, th)
    TN = get_true_neg(y, pred, th)
    FN = get_false_pos(y, pred, th)
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    return accuracy

def get_sensitivity(y, pred, th=0.5):

    """
    Function to calculate the sensitivity.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """

    sensitivity = 0.0
    TP = get_true_pos(y,pred,th)
    FN = get_false_neg(y,pred,th)
    sensitivity = TP/(TP+FN)
    return sensitivity

def get_specificity(y, pred, th=0.5):

    """
    Function to calculate the specificity.
    
    Inputs
    y: Ground truth labels
    pred: Predicted labels
    th: Classification threshold
    """
    
    specificity = 0.0
    TN = get_true_neg(y,pred,th)
    FP = get_false_pos(y,pred,th)
    specificity = TN/(TN+FP)
    return specificity 

# Preprocessing

In [ ]:
print(X_train.shape) 
print(X_test.shape) 

In [ ]:
print(X_train.min(),X_test.min())
print(X_train.max(),X_test.max())
print(X_train.shape,X_test.shape)

In [ ]:
X_train_, y_train_ = min_max_preprocessing(X_train, y_train)
X_test_, y_test_ = min_max_preprocessing(X_test, y_test) 

In [ ]:
print(X_train_.min(),X_test_.min())
print(X_train_.max(),X_test_.max())
print(X_train_.shape,X_test_.shape)  
print(y_train_.shape,y_test_.shape) 

In [ ]:
X_train = X_train_
X_test  = X_test_
y_train = y_train_
y_test  = y_test_ 

# TPU init

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu) 

# Functions used to define and work with models

In [ ]:
###############################################VGG16###############################################
def vgg16_1canal():
    pre_trained_model = tf.keras.applications.VGG16(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:2]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[2:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))

    x = Conv2D(3,(3,3),padding='same')(input_tensor)    
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model VGG16 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################VGG19###############################################
def vgg19_1canal():
    pre_trained_model = tf.keras.applications.VGG19(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:4]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[4:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)    
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model VGG19 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model

###############################################ResNet50###############################################
def resNet50_1canal():
    pre_trained_model = tf.keras.applications.ResNet50(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:100]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[100:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)   
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model ResNet50 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################ResNet152V2###############################################
def ResNet152V2_1canal():
    pre_trained_model = tf.keras.applications.ResNet152V2(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:4]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[4:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)   
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model ResNet152V2 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################InceptionV3###############################################
def InceptionV3_1canal():
    pre_trained_model = tf.keras.applications.InceptionV3(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:4]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[4:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)   
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model InceptionV3 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model
    

###############################################InceptionResNetV2###############################################
def InceptionResnetV2_1canal():
    pre_trained_model = tf.keras.applications.InceptionResNetV2(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:180]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[180:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))

    x = Conv2D(3,(3,3),padding='same')(input_tensor)    
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model InceptionResNetV2 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################MobileNetV2###############################################
def MobileNetV2_1canal():
    pre_trained_model = tf.keras.applications.MobileNetV2(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:180]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[180:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)    
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model MobileNetV2 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################DenseNet201###############################################
def DenseNet201_1canal():
    pre_trained_model = tf.keras.applications.DenseNet201(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:180]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[180:]:
       layer.trainable = True

    input_tensor = Input(shape=(224,224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)    
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
    
    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model DenseNet201 Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model


###############################################NASNetMobile###############################################
def NASNetMobile_1canal():
    pre_trained_model = tf.keras.applications.NASNetMobile(
                    include_top=False,
                    weights= 'imagenet',
                    input_shape = (224, 224, 3)
                    )
    for layer in pre_trained_model.layers[:180]: 
       layer.trainable = False
    for layer in pre_trained_model.layers[180:]:
       layer.trainable = True

    input_tensor = Input(shape=(224, 224,1))
    
    x = Conv2D(3,(3,3),padding='same')(input_tensor)   
    x = pre_trained_model(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)  
    layers = tf.keras.layers.Flatten()(x)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(1024 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense( 512,activation="relu")(layers)
    layers = tf.keras.layers.Dense( 64,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)

    model = tf.keras.Model(inputs = input_tensor, outputs=predictions)
    optimizer=tf.keras.optimizers.Adam(lr=0.0001) 
    print("Model NASNetMobile Generated")
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model 

In [ ]:
def Final_Results_Test(PATH_trained_models,model_):
    global AccTest
    global LossTest
    AccTest = []
    LossTest= [] 
    B_accuracy = 0 
    for filename in sorted(os.listdir(PATH_trained_models)):
        if filename != ('train') and filename != ('validation'):
            print(filename)
            with tpu_strategy.scope(): 
                 _model = model_
            _model.load_weights(PATH_trained_models+'/'+filename)
            loss,accuracy = _model.evaluate(X_test, y_test, verbose=0)
            print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}'+'\n')

            BandAccTest  = accuracy
            BandLossTest = loss
            AccTest.append(BandAccTest)    
            LossTest.append(BandLossTest)  
            
            if accuracy > B_accuracy:
                B_accuracy = accuracy
                B_loss = loss
                B_name = filename
    
    print("\n\nBest")
    print(B_name)
    print(f'Loss={B_loss:.4f} y Accuracy={B_accuracy:0.4f}'+'\n')
    return PATH_trained_models+'/'+B_name

def Final_Results_Train(PATH_trained_models,model_):
    global AccTrain
    global LossTrain
    AccTrain = []
    LossTrain = [] 
    B_accuracy = 0 
    for filename in sorted(os.listdir(PATH_trained_models)):
        if filename != ('train') and filename != ('validation'):
            print(filename)
            with tpu_strategy.scope(): 
                 _model = model_
            _model.load_weights(PATH_trained_models+'/'+filename)
            loss,accuracy = _model.evaluate(X_train, y_train, verbose=0)
            print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}'+'\n')

            BandAccTrain  = accuracy
            BandLossTrain = loss
            AccTrain.append(BandAccTrain)    
            LossTrain.append(BandLossTrain)  
            
            if accuracy > B_accuracy:
                B_accuracy = accuracy
                B_loss = loss
                B_name = filename
    
    print("\n\nBest")
    print(B_name)
    print(f'Loss={B_loss:.4f} y Accuracy={B_accuracy:0.4f}'+'\n')

def graphics(AccTest, AccTrain, LossTest, LossTrain, model_name, path_img_base, model_):
    if not os.path.exists(path_img_base+"/"+model_name):
       os.makedirs(path_img_base+"/"+model_name)
    with tpu_strategy.scope():  
        model = model_
    
    lossTEST,accuracyTEST   = model.evaluate(X_test, y_test,verbose=None)
    lossTRAIN,accuracyTRAIN = model.evaluate(X_train, y_train,verbose=None)
    
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(10, 10))
        plt.plot(np.concatenate([np.array([accuracyTRAIN]),np.array(AccTrain)],axis=0))
        plt.plot(np.concatenate([np.array([accuracyTEST]),np.array(AccTest)],axis=0)) #Test
        plt.title('Accuracy Vs Epoch')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/'+model_name+'/Accuracy_'+model_name+'.eps', format='eps')
        plt.savefig(path_img_base+'/'+model_name+'/Accuracy_'+model_name+'.svg', format='svg')
        plt.savefig(path_img_base+'/'+model_name+'/Accuracy_'+model_name+'.pdf', format='pdf')     
        plt.show()
        
        plt.figure(figsize=(10, 10))
        plt.plot(np.concatenate([np.array([lossTRAIN]),np.array(LossTrain)],axis=0))
        plt.plot(np.concatenate([np.array([lossTEST]),np.array(LossTest)],axis=0)) #Test
        plt.title('Loss Vs Epoch')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.grid('on')
        plt.savefig(path_img_base+'/'+model_name+'/Loss_'+model_name+'.eps', format='eps')
        plt.savefig(path_img_base+'/'+model_name+'/Loss_'+model_name+'.svg', format='svg')
        plt.savefig(path_img_base+'/'+model_name+'/Loss_'+model_name+'.pdf', format='pdf') 
        plt.show()

def trainTPU(path_model, epochs, model_Name, m_selected="VGG16"):
    global model_name
    start_time = tm.time()
    model_name = model_Name
    path_log_base = path_model+'/'+model_Name
    if not os.path.exists(path_log_base):
        os.makedirs(path_log_base)

    with tpu_strategy.scope():
      if m_selected == "VGG16":
        model = vgg16_1canal()
      elif m_selected == "VGG19":
        model = vgg19_1canal()
      elif m_selected == "ResNet50":
        model = resNet50_1canal()
      elif m_selected == "ResNet152V2":
        model = ResNet152V2_1canal()
      elif m_selected == "InceptionV3":
        model = InceptionV3_1canal()
      elif m_selected == "InceptionResnetV2":
        model = InceptionResnetV2_1canal()
      elif m_selected == "MobileNetV2":
        model = MobileNetV2_1canal()
      elif m_selected == "DenseNet201":
        model = DenseNet201_1canal()
      else:
        model = NASNetMobile_1canal()

    epoch_ = 1
    for epoch in range(epochs):
        epoch=epoch+1
        print("epoch ",epoch)
        model.fit(X_train,y_train,validation_data=(X_test,y_test), batch_size=64*4, epochs=epoch_, verbose=1) 
        model.save_weights(path_model+'/'+model_name+'/'+str(epoch).zfill(4)+'.hdf5', overwrite=True) 

    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME) 

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred):
  smooth = 1
  intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
  return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
  return 1-dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        smooth = 1
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def focal_loss(y_true, y_pred):
    alpha=0.25
    gamma=2
    def focal_loss_with_logits(logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)
        return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(-logits)) * (weight_a + weight_b) + logits * weight_b

    y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    logits = tf.math.log(y_pred / (1 - y_pred))
    loss = focal_loss_with_logits(logits=logits, targets=y_true, alpha=alpha, gamma=gamma, y_pred=y_pred)
    return tf.reduce_mean(loss) 

In [ ]:
def Unet_3():
  inputs = Input(shape=(224, 224, 1), name='input')

  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.Dropout(0.1)(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.1)(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.Dropout(0.3)(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.1)(c8)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
  
  u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = tf.keras.layers.Dropout(0.1)(c9)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  print("Model U-Net Generated")
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,]) 
  return model 

In [ ]:
SEGmodel = Unet_3()

In [ ]:
SEGmodel.load_weights(".../Unet_finetune_v3.h5") 

In [ ]:
print(X_train.shape,X_test.shape) 

In [ ]:
SEGX_train= np.expand_dims(X_train,axis=3)
SEGX_test = np.expand_dims(X_test,axis=3)

In [ ]:
print(SEGX_train.shape,SEGX_test.shape) 

In [ ]:
X_train_masks=SEGmodel.predict(SEGX_train) 
X_test_masks=SEGmodel.predict(SEGX_test) 

In [ ]:
X_train_masks=np.round(X_train_masks) 
X_test_masks=np.round(X_test_masks) 

In [ ]:
X_train_masks = X_train_masks.astype(np.uint8) 
X_test_masks = X_test_masks.astype(np.uint8) 

In [ ]:
path_model = "./DATA/"
if not os.path.exists(path_model):
  os.makedirs(path_model) 

In [ ]:
np.save('./DATA/X_train_masks_seed#',X_train_masks)
np.save('./DATA/X_test_masks_seed#',X_test_masks) 

In [ ]:
X_train_masks = np.load('./DATA/X_train_masks_seed#.npy')
X_test_masks  = np.load('./DATA/X_test_masks_seed#.npy') 

In [ ]:
X_train = X_train*X_train_masks[:,:,:,0]
X_test  = X_test*X_test_masks[:,:,:,0] 

# Model training and evaluation

In [ ]:
path_model = ".../Results/Models"
if not os.path.exists(path_model):
  os.makedirs(path_model) 

In [ ]:
path_img_base = ".../Results/Figure"
if not os.path.exists(path_img_base):
  os.makedirs(path_img_base) 

In [ ]:
model_Name = "_InceptionV3" 
trainTPU(path_model=path_model, epochs=50, model_Name = model_Name, m_selected="InceptionV3") #Choose one model 

In [ ]:
path_model = ".../Results/Models"
path_img_base = ".../Results/Figure" 

In [ ]:
M=DenseNet201_1canal()
model_Name="_InceptionV3" 

In [ ]:
best=Final_Results_Test(path_model+"/"+model_Name, M) 

In [ ]:
model=M
model.load_weights(best)
metrics_1(model,X_test,y_test) 

In [ ]:
Final_Results_Train(path_model+"/"+model_Name, M) 

In [ ]:
graphics(AccTest, AccTrain, LossTest, LossTrain, model_Name, path_img_base, M) 